<a href="https://colab.research.google.com/github/DmitryIo/sentiment_twitter/blob/master/SENTIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
#@title connecting to google drive(skipped)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#@title reading train/test from google drive
X_train = pd.read_csv('/content/drive/My Drive/banana projects/sentiment analyz/X_train.csv')
X_test = pd.read_csv('/content/drive/My Drive/banana projects/sentiment analyz/X_test (1).csv')
y_train = pd.read_csv('/content/drive/My Drive/banana projects/sentiment analyz/y_train.csv') 
sample = pd.read_csv('/content/sample_random.csv')

### Preprocessing

In [ ]:

# функция для удаления URL из строк

import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http|http |https | https| http)?(:| :|: )\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)


print( remove_urls("this is a test https : //sdfs.sdfsdf.com/sdfsdf/sdfsdf/sd/sdfsdfs?bob=%20tree&jef=man lets see this too https://sdfsdf.fdf.com/sdf/f end"))

this is a test  lets see this too end


In [ ]:

# Стемминг(т.е. выделение корня слова. Обычно используется вместо лемматизации)
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
# Лемматизация 
sentence = "The striped bats are hanging on their feet for best"
def lemSentence(sentence): 
  lemmatizer = WordNetLemmatizer()
  word_list = nltk.word_tokenize(sentence)
  return ' '.join([lemmatizer.lemmatize(w) for w in word_list])
lemSentence(sentence)

'The striped bat are hanging on their foot for best'

In [ ]:
# Удаление обращений в строке(в твиттере строка начинается с @'имя пользователя' 'текст поста')
def remove_reply(vTEXT):
    vTEXT = re.sub(r"(@(\w+)|@ (\w+)| @(\w+))", '', vTEXT, flags=re.MULTILINE)
    return ' '.join([w for w in vTEXT.split() if w != ' '])


In [ ]:
# удаление пунктуации
def punctuation(sentence):
  punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
  return ''.join([i for i in sentence if not i in punctuations])


# **Preprocessing**

In [ ]:
X_train['text_preprocessed'] = X_train['SentimentText'].apply(remove_reply)
X_test['text_preprocessed'] = X_test['SentimentText'].apply(remove_reply)
X_train['text_preprocessed'] = X_train['text_preprocessed'].apply(remove_urls)
X_test['text_preprocessed'] = X_test['text_preprocessed'].apply(remove_urls)
X_train['text_preprocessed'] = X_train['text_preprocessed'].apply(punctuation)
X_test['text_preprocessed'] = X_test['text_preprocessed'].apply(punctuation)
X_train['text_preprocessed'] = X_train['text_preprocessed'].apply(lemSentence)
X_test['text_preprocessed'] = X_test['text_preprocessed'].apply(lemSentence)


In [ ]:
corpus = X_train['text_preprocessed'].values
corpus_test = X_test['text_preprocessed'].values

### Vectorizing and validation

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(corpus, y_train['Sentiment'], test_size=0.3, random_state=42)


vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 7))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_valid_vectorized = vectorizer.transform(X_valid)



In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train_vectorized, y_train)

In [ ]:
y_pred = log_reg.predict(X_valid_vectorized)

**Checking of saving distribution of targets:**

In [ ]:
pd.Series(y_pred).value_counts()

1    16388
0    10609
dtype: int64

In [ ]:
y_valid.value_counts()

1    15259
0    11738
Name: Sentiment, dtype: int64

In [ ]:
accuracy_score(y_valid, y_pred)

0.7751231618327963

### Final submit

In [ ]:
y_train = pd.read_csv('/content/drive/My Drive/banana projects/sentiment analyz/y_train.csv') 

In [ ]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 7))
X_train_vectorized = vectorizer.fit_transform(corpus)
X_test_vectorized = vectorizer.transform(corpus_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
log_reg=LogisticRegression()
log_reg_cv=GridSearchCV(log_reg,grid,cv=5)
log_reg_cv.fit(X_train_vectorized, y_train['Sentiment'])

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(X_train_vectorized, y_train['Sentiment'])

In [ ]:
log_reg_cv.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [ ]:
log_reg = LogisticRegression(C=1.0, penalty='l2')
log_reg.fit(X_train_vectorized, y_train['Sentiment'])

In [ ]:
y_preds = log_reg.predict(X_test_vectorized)

In [ ]:
sample['Sentiment'] = y_preds

sample.to_csv('baseline_tfidf.csv', index=False)